In [2]:
import numpy as np
import numpy.linalg as lg 
from  scipy.io import mmread
from scipy.sparse.linalg import cg
from scipy.linalg import blas
from tabulate import tabulate
A1=np.array(mmread('/content/bcsstk14.mtx.gz').todense())
# A1.shape=(1806, 1806) real symmetric positive definite
A2=np.array(mmread('/content/bcsstm19.mtx.gz').todense())
# A2.shape=(817,817) real symmetric positive definite
A3=np.array(mmread('/content/bcsstm20.mtx.gz').todense())
# A3.shape=(485,485) real symmetric positive definite
A4=np.array(mmread('/content/bcsstm24.mtx.gz').todense())
# A4.shape=(3562,3562) real symmetric positive definite
B1=np.array(mmread('/content/bcsstm21.mtx.gz').todense())
B2=np.array(mmread('/content/bcsstk16.mtx.gz').todense())
B3=np.array(mmread('/content/bcsstk17.mtx.gz').todense())
B4=np.array(mmread('/content/bcsstk18.mtx.gz').todense())
C1=np.array(mmread('/content/bcsstm22.mtx.gz').todense())
C2=np.array(mmread('/content/bcsstk27.mtx.gz').todense())
C3=np.array(mmread('/content/bcsstm26.mtx.gz').todense())

# **The preconditionned Conjugate Gradient:**

# The Jacobi:

In [3]:
def cgp_iter(A, b,x0,maxiter,tol,M):
    num_iters = 0

    def callback(xk):
        nonlocal num_iters
        num_iters += 1
    return cg(A, b, callback=callback,tol = tol,maxiter=maxiter,M=lg.inv(M)),num_iters


def Preconditionned_Conjugate_Gradient_Algorithm(A,b,x,itermax,tol):
    M=np.tril(A, k=0)
    r = b- blas.dgemv(1.0, A, x)  
    z = p = blas.dgemv(1.0,lg.inv(M) , r)   
    k = 0 
    
    while  blas.dnrm2(z) > tol :   
        alpha1 =  blas.ddot(r, z)/ ( blas.ddot(blas.dgemv(1.0, A, p), p))                
        x = x + alpha1*p
        rpr=r.copy()
        
        r = rpr - blas.dgemv(alpha1, A, p)     
        zpr = z.copy()
        z =  blas.dgemv(1.0,lg.inv(M) , r)             
        beta1 =  blas.ddot(r, z)/blas.ddot(rpr, zpr)    
        p = z + beta1*p
        k = k + 1
    return x,k


In [ ]:
matricies=[A1,A4,C3]
tol=1e-5
itermax = 1000

In [ ]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  M=np.diag(np.diag(matricies[i]))
  b=np.ones(n)
  x = np.zeros(n)
  sol  = lg.solve(matricies[i], b)
  itersc = cgp_iter(matricies[i],b,x,itermax,tol,M)[1]  
  y=cg(matricies[i],b,x,tol,itermax,M=lg.inv(M))[0]
  sc_err=np.linalg.norm(y-sol)
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  x,iterm  = Preconditionned_Conjugate_Gradient_Algorithm(matricies[i],b,x,itermax,tol)
  err=lg.norm(x-sol)
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(tol)
  l.append(iterm)
  l.append(res)
  l.append(err)
  l.append(itersc)
  l.append(sc_res)
  l.append(sc_err)
  my_data.append(l)
# create header
head = ['matricies','tol','Our_nbr_iter','res','Our_err','scipy_nbr_iter','sc_res','scipy_err']
  
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+--------------+-------+----------------+---------------+-------------+------------------+-------------+-------------+
| matricies    |   tol |   Our_nbr_iter |           res |     Our_err |   scipy_nbr_iter |      sc_res |   scipy_err |
+==============+=======+================+===============+=============+==================+=============+=============+
| (1806, 1806) | 1e-05 |             38 | 254.592       | 0.000876595 |              370 | 0.000411509 | 2.05657e-10 |
+--------------+-------+----------------+---------------+-------------+------------------+-------------+-------------+
| (3562, 3562) | 1e-05 |              1 |   2.15852e-15 | 0           |                1 | 2.15852e-15 | 0           |
+--------------+-------+----------------+---------------+-------------+------------------+-------------+-------------+
| (1922, 1922) | 1e-05 |              1 |   1.73422e-15 | 0           |                1 | 9.71175e-15 | 1.25845e-10 |
+--------------+-------+----------------+-------

# The symmetric Gauss-Seidel:

In [ ]:
def cgp_iter(A, b,x0,maxiter,tol,M):
    num_iters = 0

    def callback(xk):
        nonlocal num_iters
        num_iters += 1
    return cg(A, b, callback=callback,tol = tol,maxiter=maxiter,M=lg.inv(M)),num_iters


def Preconditionned_Conjugate_Gradient_Algorithm(A,b,x,itermax,tol):
    a=np.tril(A, k=0)
    c=np.triu(A, k=0)
    D=np.diag(np.diag(A))
    M= np.matmul(a,np.matmul(lg.inv(D),c))
    r = b-blas.dgemv(1.0, A, x)       
    z = p = blas.dgemv(1.0,lg.inv(M) , r)         
    k = 0 
    
    while blas.dnrm2(z) > tol :
        alpha1 =  blas.ddot(r, z)/ ( blas.ddot(blas.dgemv(1.0, A, p), p))      
        x = x + alpha1*p
        rpr=r.copy()
        
        r = rpr - blas.dgemv(alpha1, A, p)      
        zpr = z.copy()
        z =  blas.dgemv(1.0,lg.inv(M) , r)             
        beta1 =  blas.ddot(r, z)/blas.ddot(rpr, zpr)      
        p = z + beta1*p
        k = k + 1
    return x,k


In [ ]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  a=np.tril(matricies[i], k=0)
  c=np.triu(matricies[i], k=0)
  D=np.diag(np.diag(matricies[i]))
  M= np.matmul(a,np.matmul(lg.inv(D),c))
  b=np.ones(n)
  x = np.zeros(n)
  sol  = lg.solve(matricies[i], b)
  itersc = cgp_iter(matricies[i],b,x,itermax,tol,M)[1]  
  y=cg(matricies[i],b,x,tol,itermax,M=lg.inv(M))[0]
  sc_err=np.linalg.norm(y-sol)
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  x,iterm  = Preconditionned_Conjugate_Gradient_Algorithm(matricies[i],b,x,itermax,tol)
  err=lg.norm(x-sol)
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(tol)
  l.append(iterm)
  l.append(res)
  l.append(err)
  l.append(itersc)
  l.append(sc_res)
  l.append(sc_err)
  my_data.append(l)
# create header
head = ['matricies','tol','Our_nbr_iter','res','Our_err','scipy_nbr_iter','sc_res','scipy_err']
  
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+--------------+-------+----------------+---------------+-------------+------------------+-------------+-------------+
| matricies    |   tol |   Our_nbr_iter |           res |     Our_err |   scipy_nbr_iter |      sc_res |   scipy_err |
+==============+=======+================+===============+=============+==================+=============+=============+
| (1806, 1806) | 1e-05 |             19 | 162.565       | 0.000524533 |              175 | 0.000389337 | 3.20373e-10 |
+--------------+-------+----------------+---------------+-------------+------------------+-------------+-------------+
| (3562, 3562) | 1e-05 |              1 |   3.00376e-15 | 6.45239e-09 |                1 | 3.00376e-15 | 6.45239e-09 |
+--------------+-------+----------------+---------------+-------------+------------------+-------------+-------------+
| (1922, 1922) | 1e-05 |              1 |   2.90361e-15 | 4.16627e-11 |                1 | 9.00103e-15 | 1.18828e-10 |
+--------------+-------+----------------+-------